In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob

In [ ]:
# paths = [
#     "../results/bo/ackley53/*_iters.csv",
#      "../results/*dim53Ackley*_iters.csv"
# ]
paths = [
    "../results/*/*/*_iters.csv"
]
res = pd.concat(
    pd.read_csv(f) for p in paths for f in glob(p)
)
res.head()

In [ ]:
res['iter_total_time'] = res['iter_eval_time'] + res['iter_model_time']

In [ ]:
resm = res.groupby(['approach', 'problem', 'iter_idx']).agg(
    iter_model_time_mean=pd.NamedAgg(column='iter_model_time', aggfunc=np.mean),
    iter_model_time_std=pd.NamedAgg(column='iter_model_time', aggfunc=np.std),
    iter_model_time_q05=pd.NamedAgg(column='iter_model_time', aggfunc=lambda x: np.quantile(x, 0.05)),
    iter_model_time_q95=pd.NamedAgg(column='iter_model_time', aggfunc=lambda x: np.quantile(x, 0.95)),
    iter_total_time_mean=pd.NamedAgg(column='iter_total_time', aggfunc=np.mean),
    iter_total_time_std=pd.NamedAgg(column='iter_total_time', aggfunc=np.std),
    iter_total_time_q05=pd.NamedAgg(column='iter_total_time', aggfunc=lambda x: np.quantile(x, 0.05)),
    iter_total_time_q95=pd.NamedAgg(column='iter_total_time', aggfunc=lambda x: np.quantile(x, 0.95)),
    iter_best_fitness_mean=pd.NamedAgg(column='iter_best_fitness', aggfunc=np.mean),
    iter_best_fitness_std=pd.NamedAgg(column='iter_best_fitness', aggfunc=np.std),
    iter_best_fitness_q05=pd.NamedAgg(column='iter_best_fitness', aggfunc=lambda x: np.quantile(x, 0.05)),
    iter_best_fitness_q95=pd.NamedAgg(column='iter_best_fitness', aggfunc=lambda x: np.quantile(x, 0.95)),
)
resm = resm.reset_index()
resm.head()

In [ ]:
resm.head()

In [ ]:
resps = resm.groupby('problem')
# margintype = 'std'
margintype = 'q5-95'

for problem, respsi in resps:
    fig = plt.figure()
    ax = fig.subplots()

    ax.set_title(problem)
    ax.set_xscale('log')
    if "SteelFoldPlate" in problem:
        ax.set_yscale('log')
    ax.set_ylabel("Fitness")
    ax.set_xlabel("Time (s)")

    for (_, sr) in respsi.groupby('approach'):
        # x = np.cumsum(sr['iter_model_time_mean'])
        x = np.cumsum(sr['iter_total_time_mean'])
        y = sr['iter_best_fitness_mean']
        if "SteelFoldPlate" in problem:
            y = y + 1.0
        ax.plot(x, y, label=sr['approach'].iloc[0])

        plot_margin = True
        # xstd = sr['iter_model_time_std']
        if margintype == 'std':
            # print("Plotting standard deviation")
            xstd = sr['iter_total_time_std']
            xmin = x - xstd
            xmax = x + xstd
            ystd = sr['iter_best_fitness_std']
            ymin = y - ystd
            ymax = y + ystd
        elif margintype == 'q5-95':
            # print("Plotting quantiles.")
            xmin = sr['iter_total_time_q05']
            xmax = sr['iter_total_time_q95']
            ymin = sr['iter_best_fitness_q05']
            ymax = sr['iter_best_fitness_q95']
        else:
            plot_margin = False

        if plot_margin:
            # print(f"Adding margin.\n{ymin}\n{ymax}")
            ax.fill_between(x, ymin, ymax, alpha=0.1)

    ax.legend()